In [ ]:
from avapi.carla import CarlaScenesManager

data_dir = "/data/shared/CARLA/multi-agent-aerial/"
CSM = CarlaScenesManager(data_dir=data_dir)
CDM = CSM.get_scene_dataset_by_index(0)

In [ ]:
from avstack.maskfilters import box_in_fov


# get data for the frame
frame_idx = 10
agent_idx = 7
frame = CDM.get_frames(sensor="camera-0", agent=0)[frame_idx]
agent = CDM.get_agents(frame=frame)[agent_idx]
agent_ref = agent.as_reference()
objects_global = CDM.get_objects_global(frame=frame, ignore_static_agents=True)
img = CDM.get_image(frame=frame, sensor="camera-0", agent=agent_idx)
depth_img = CDM.get_depth_image(frame=frame, sensor="depthcamera-0", agent=agent_idx)
objs_img = CDM.get_objects(frame=frame, sensor="camera-0", agent=agent_idx)

In [ ]:
import numpy as np


obj = objs_img[1]
box_2d = obj.box.project_to_2d_bbox(
    depth_img.calibration, check_reference=True
).squeeze(
    depth_img.calibration.height,
    depth_img.calibration.width,
    inplace=False,
)
depths = depth_img.depths[
    int(box_2d.ymin) : int(box_2d.ymax), int(box_2d.xmin) : int(box_2d.xmax)
]

# assume depth is the last coordinate along the position (img coords)
centered_depths = np.reshape(depths, -1) - (abs(obj.position[2]) - obj.box.l / 2)

d_close = 3.0
frac_viewable = sum(np.abs(centered_depths) < d_close) / len(centered_depths)
print(frac_viewable)

In [ ]:
from avapi.visualize.snapshot import show_image_with_boxes

# show_image_with_boxes(img=img, boxes=objs_img[1], inline=True)
show_image_with_boxes(img=img, boxes=objs_img, inline=True)